# ထုတ်လုပ်မှုအတွက် AutoGen အေးဂျင့်များ: စောင့်ကြည့်မှုနှင့် အကဲဖြတ်မှု

ဒီသင်ခန်းစာမှာ **[Autogen agents](https://github.com/microsoft/autogen)** ရဲ့ အတွင်းပိုင်းအဆင့်များ (traces) ကို **စောင့်ကြည့်နည်း**နဲ့ [Langfuse](https://langfuse.com) ကို အသုံးပြုပြီး **စွမ်းဆောင်ရည်ကို အကဲဖြတ်နည်း**ကို လေ့လာသင်ယူပါမယ်။

ဒီလမ်းညွှန်မှာ **အွန်လိုင်း**နဲ့ **အော့ဖ်လိုင်း** အကဲဖြတ်မှုများကို ဖော်ပြထားပြီး အေးဂျင့်များကို အမြန်ဆုံးနဲ့ ယုံကြည်စိတ်ချစွာ ထုတ်လုပ်မှုအဆင့်သို့ ရောက်အောင် ကူညီပေးပါမယ်။

**AI အေးဂျင့် အကဲဖြတ်မှု အရေးကြီးတဲ့အကြောင်း:**
- တာဝန်များ မအောင်မြင်တာ သို့မဟုတ် မကျေနပ်ဖွယ် ရလဒ်များ ထွက်လာတဲ့အခါ ပြဿနာများကို ပြုပြင်နိုင်ရန်
- အချိန်နှင့်တပြေးညီ ကုန်ကျစရိတ်နဲ့ စွမ်းဆောင်ရည်ကို စောင့်ကြည့်ရန်
- ဆက်လက်တိုးတက်မှုအတွက် အမြဲတမ်းတုံ့ပြန်ချက်များဖြင့် ယုံကြည်စိတ်ချမှုနှင့် လုံခြုံမှုကို တိုးတက်စေရန်


## အဆင့် ၁: ပတ်ဝန်းကျင်အပြောင်းအလဲများကို သတ်မှတ်ပါ

[Langfuse Cloud](https://cloud.langfuse.com/) တွင် စာရင်းသွင်းခြင်း သို့မဟုတ် [Langfuse ကို ကိုယ်တိုင်တပ်ဆင်ခြင်း](https://langfuse.com/self-hosting) မှတဆင့် Langfuse API key များကို ရယူပါ။

_**မှတ်ချက်:** ကိုယ်တိုင်တပ်ဆင်သူများသည် [Terraform modules](https://langfuse.com/self-hosting/azure) ကို အသုံးပြု၍ Azure ပေါ်တွင် Langfuse ကို တပ်ဆင်နိုင်သည်။ အခြားနည်းလမ်းအဖြစ် [Helm chart](https://langfuse.com/self-hosting/kubernetes-helm) ကို အသုံးပြု၍ Kubernetes ပေါ်တွင် Langfuse ကို တပ်ဆင်နိုင်ပါသည်။_


In [5]:
import os

# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..." 
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..." 
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com" # 🇪🇺 EU region
# os.environ["LANGFUSE_HOST"] = "https://us.cloud.langfuse.com" # 🇺🇸 US region

ပတ်ဝန်းကျင်အပြောင်းအလဲများကို သတ်မှတ်ပြီးနောက်၊ ယခု Langfuse client ကို စတင်နိုင်ပါပြီ။ `get_client()` သည် ပတ်ဝန်းကျင်အပြောင်းအလဲများတွင် ပေးထားသော အတည်ပြုချက်များကို အသုံးပြု၍ Langfuse client ကို စတင်သည်။


In [6]:
from langfuse import Langfuse
 
# Filter out Autogen OpenTelemetryspans
langfuse = Langfuse(
    blocked_instrumentation_scopes=["autogen SingleThreadedAgentRuntime"]
)
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")

Langfuse client is authenticated and ready!


## အဆင့် ၂ - OpenLit Instrumentation ကို စတင်ခြင်း

ယခုအချိန်တွင် [OpenLit](https://github.com/openlit/openlit) instrumentation ကို စတင်မည်ဖြစ်သည်။ OpenLit သည် AutoGen လုပ်ဆောင်မှုများကို အလိုအလျောက် ဖမ်းယူပြီး OpenTelemetry (OTel) spans များကို Langfuse သို့ တင်ပို့ပေးပါသည်။


In [7]:
import openlit
 
# Initialize OpenLIT instrumentation. The disable_batch flag is set to true to process traces immediately.
openlit.init(tracer=langfuse._otel_tracer, disable_batch=True, disabled_instrumentors=["mistral"])

## အဆင့် ၃ - သင့်အေးဂျင့်ကို အလုပ်လုပ်စေပါ

ယခုအချိန်တွင် ကျွန်ုပ်တို့၏ ပစ္စည်းကိရိယာများကို စမ်းသပ်ရန် အဆင့်ဆင့် အေးဂျင့်တစ်ခုကို ပြင်ဆင်ပါမည်။


In [2]:
import os

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

In [3]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
        "structured_output": False
    },
)

In [8]:
# 🍴 Agent 1 – proposes ONE healthy meal idea each turn
meal_planner_agent = AssistantAgent(
    "meal_planner_agent",
    model_client=client,
    description="A seasoned meal-planning coach who suggests balanced meals.",
    system_message="""
    You are a Meal-Planning Assistant with a decade of experience helping busy people prepare meals.
    Goal: propose the single best meal (breakfast, lunch, or dinner) given the user's context.
    Each response must contain ONLY one complete meal idea (title + very brief component list) — no extras.
    Keep it concise: skip greetings, chit-chat, and filler.
    """,
)

# 🥗 Agent 2 – checks nutritional quality & variety
nutritionist_agent = AssistantAgent(
    "nutritionist_agent",
    model_client=client,
    description="A registered dietitian ensuring meals meet nutritional standards.",
    system_message="""
    You are a Nutritionist focused on whole-food, macro-balanced eating.
    Evaluate the meal_planner_agent’s recommendation.
    If the meal is nutritionally sound, sufficiently varied, and portion-appropriate, respond with 'APPROVE'.
    Otherwise, give high-level guidance on how to improve it (e.g. 'add a plant-based protein') — do NOT provide a full alternative recipe.
    """,
)

In [9]:
# ✅ Chat stops once the nutritionist says APPROVE
termination = TextMentionTermination("APPROVE")

# 🔄 Alternate turns between the two agents until termination
team = RoundRobinGroupChat(
    [meal_planner_agent, nutritionist_agent],
    termination_condition=termination,
)

# Example kickoff
user_input = "I'm looking for a quick, delicious dinner I can prep after work. I have 30 minutes and minimal clean-up is ideal."

In [ ]:
with langfuse.start_as_current_span(name="create_meal_plan") as span:
    async for message in team.run_stream(task=user_input):
        if isinstance(message, TaskResult):
            print("Stop Reason:", message.stop_reason)
        else:
            print(message)

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

# Flush the trace to Langfuse for short-lived environments such as Jupyter Notebooks
langfuse.flush()

### အကြောင်းအရာဖွဲ့စည်းမှု

Langfuse သည် **trace** တစ်ခုကို မှတ်တမ်းတင်ပြီး၊ ၎င်းတွင် **spans** များပါဝင်သည်။ Span များသည် သင့် agent ၏ logical အဆင့်တစ်ခုချင်းစီကို ကိုယ်စားပြုသည်။ ဤနေရာတွင် trace တွင် အထွေထွေ agent ၏ လုပ်ဆောင်မှုနှင့် အောက်ပါ sub-spans များပါဝင်သည်-
- အစားအသောက်အစီအစဉ်ရေးဆွဲသည့် agent
- အာဟာရပညာရှင်များ

ဤ span များကို စစ်ဆေးခြင်းဖြင့် အချိန်ကို မည်သို့အသုံးပြုထားသည်ကို၊ token များကို မည်မျှအသုံးပြုထားသည်ကို စသဖြင့် တိတိကျကျ ကြည့်ရှုနိုင်ပါသည်-

![Langfuse တွင် Trace tree](https://langfuse.com/images/cookbook/example-autogen-evaluation/trace-tree.png)

_[Trace link ကိုကြည့်ရန်](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


## အွန်လိုင်းအကဲဖြတ်ခြင်း

အွန်လိုင်းအကဲဖြတ်ခြင်းဆိုသည်မှာ အေးဂျင့်ကို တိုက်ရိုက်၊ အမှန်တကယ်အသုံးပြုနေသော ပတ်ဝန်းကျင်တွင် အကဲဖြတ်ခြင်းကို ဆိုလိုသည်။ ဒါဟာ အေးဂျင့်ရဲ့ စစ်မှန်တဲ့အသုံးပြုသူအပြောအဆိုများကို စောင့်ကြည့်ပြီး အကျိုးအမြတ်များကို ဆက်လက်လေ့လာသုံးသပ်ခြင်းဖြစ်ပါတယ်။

### ထုတ်လုပ်မှုအတွင်း စောင့်ကြည့်သင့်သော သာမန်အချက်အလက်များ

1. **ကုန်ကျစရိတ်များ** — အင်စထရူမန်တေးရှင်းက token အသုံးပြုမှုကို ဖမ်းယူပြီး token တစ်ခုချင်းစီအတွက် စျေးနှုန်းသတ်မှတ်ခြင်းအားဖြင့် ခန့်မှန်းကုန်ကျစရိတ်များကို ပြောင်းလဲနိုင်သည်။
2. **နောက်ကျမှု** — တစ်ခုချင်းစီအဆင့်ကို ပြီးမြောက်ရန် သို့မဟုတ် အပြေးလုံးကို ပြီးမြောက်ရန် ကြာချိန်ကို စောင့်ကြည့်ပါ။
3. **အသုံးပြုသူတုံ့ပြန်မှု** — အသုံးပြုသူများသည် အေးဂျင့်ကို ပိုမိုကောင်းမွန်စေရန် သို့မဟုတ် ပြင်ဆင်ရန်အတွက် တိုက်ရိုက်တုံ့ပြန်မှု (လက်မောင်းတင်/လက်မောင်းချ) ပေးနိုင်သည်။
4. **LLM-as-a-Judge** — သင့်အေးဂျင့်ရဲ့ output ကို နီးစပ်သော အချိန်တွင် (ဥပမာ- အဆိုးအယုတ်မှု သို့မဟုတ် တိကျမှု စစ်ဆေးခြင်း) အကဲဖြတ်ရန် သီးခြား LLM ကို အသုံးပြုပါ။

အောက်တွင် ဤအချက်အလက်များ၏ ဥပမာများကို ပြသထားပါသည်။


#### ၁။ ကုန်ကျစရိတ်များ

အောက်တွင် `gpt-4o-mini` ကိုခေါ်ဆိုမှုများအတွက် အသုံးပြုမှုကို ပြသထားသော screenshot တစ်ခုရှိသည်။ ဤသည်မှာ ကုန်ကျစရိတ်များသော အဆင့်များကို ကြည့်ရှုပြီး သင့် agent ကို အထိရောက်ဆုံးဖြစ်စေရန် optimize ပြုလုပ်ရန် အသုံးဝင်ပါသည်။

![Costs](https://langfuse.com/images/cookbook/example-autogen-evaluation/gpt-4o-costs.png) 

_[Link to the trace](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64)_


#### ၂။ အချိန်နှောင့်နှေးမှု

အဆင့်တစ်ခုချင်းစီကို ပြီးမြောက်ရန် ယူသည့်အချိန်ကိုလည်း ကြည့်ရှုနိုင်ပါတယ်။ အောက်ပါဥပမာတွင် အားလုံးကို ပြီးမြောက်ရန် ၃ စက္ကန့်ခန့်ယူခဲ့ပြီး၊ အဆင့်တစ်ခုချင်းစီအလိုက် ခွဲခြားကြည့်နိုင်ပါတယ်။ ဒါက သင့်ကို အချိန်နှောင့်နှေးမှုရှိရာနေရာများကို ဖော်ထုတ်ပြီး သင့်အေးဂျင့်ကို အထူးပြုလုပ်ရန် ကူညီပေးနိုင်ပါတယ်။

![Latency](https://langfuse.com/images/cookbook/example-autogen-evaluation/agent-latency.png) 

_[လမ်းကြောင်းကိုကြည့်ရန်](https://cloud.langfuse.com/project/cloramnkj0002jz088vzn1ja4/traces/dac2b33e7cd709e685ccf86a137ecc64?display=timeline)_


#### 3. အသုံးပြုသူ၏ အကြောင်းပြန်ချက်

သင့် Agent ကို အသုံးပြုသူအင်တာဖေ့စ်ထဲတွင် ထည့်သွင်းထားပါက အသုံးပြုသူ၏ တိုက်ရိုက်အကြောင်းပြန်ချက်များ (ဥပမာ - chat UI ထဲတွင် thumbs-up/down) ကို မှတ်တမ်းတင်နိုင်ပါသည်။


In [10]:
from langfuse import get_client
 
langfuse = get_client()
 
# Option 1: Use the yielded span object from the context manager
with langfuse.start_as_current_span(
    name="autogen-request-user-feedback-1") as span:
    
    async for message in team.run_stream(task="Create a meal with potatoes"):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using the span object
    span.score_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC",
        comment="This was delicious, thank you"
    )
 
# Option 2: Use langfuse.score_current_trace() if still in context
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:
    # ... Autogen execution ...

    async for message in team.run_stream(task="I am allergic to gluten."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message)    
 
    # Score using current context
    langfuse.score_current_trace(
        name="user-feedback",
        value=1,
        data_type="NUMERIC"
    )

id='da068880-22ae-4f01-9f01-2bb231939089' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 43, 732669, tzinfo=datetime.timezone.utc) content='Create a meal with potatoes' type='TextMessage'
id='ad937ce4-3534-493f-824b-ca9c226b5287' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=95, completion_tokens=30) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 186423, tzinfo=datetime.timezone.utc) content='Potato and Spinach Frittata  \n- Eggs  \n- Potatoes  \n- Fresh spinach  \n- Onion  \n- Cheese (optional)  ' type='TextMessage'
id='50fd33c1-057f-49fe-afad-ee86d164296d' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=132, completion_tokens=4) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 20, 45, 581059, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
id='e371de6c-e5fc-42c1-8eda-e5b8cd5accab' source='user' models_usage=None met

In [ ]:
# Option 3: Use create_score() with trace ID (when outside context)
langfuse.create_score(
    trace_id="predefined_trace_id",
    name="user-feedback",
    value=1,
    data_type="NUMERIC",
    comment="This was correct, thank you"
)

![အသုံးပြုသူ၏တုံ့ပြန်မှုကို Langfuse တွင်ဖမ်းယူနေသည်](https://langfuse.com/images/cookbook/example-autogen-evaluation/user-feedback.png)


#### 4. အလိုအလျောက် LLM-as-a-Judge အဆင့်သတ်မှတ်ခြင်း

LLM-as-a-Judge သည် သင့်အေဂျင့်၏ output ကို အလိုအလျောက် အကဲဖြတ်ရန် နောက်ထပ်နည်းလမ်းတစ်ခုဖြစ်သည်။ သင့်အနေဖြင့် output ၏ မှန်ကန်မှု၊ အဆိပ်အတောက်ရှိမှု၊ စတိုင် သို့မဟုတ် သင့်အတွက် အရေးကြီးသော အခြားအချက်များကို တိုင်းတာရန် သီးခြား LLM ကို ခေါ်ယူနိုင်သည်။

**လုပ်ငန်းစဉ်**:
1. **အကဲဖြတ်မှုအခြေခံပုံစံ** တစ်ခုကို သတ်မှတ်ပါ၊ ဥပမာ - "စာသားမှာ အဆိပ်အတောက်ရှိမရှိ စစ်ဆေးပါ။"
2. သုံးမည့် judge-model ကို သတ်မှတ်ပါ၊ ဤကိစ္စတွင် `gpt-4o-mini` ကို Azure မှတစ်ဆင့် မေးမြန်းသည်။
3. သင့်အေဂျင့် output တစ်ခုကို ဖန်တီးတိုင်း၊ ထို output ကို "judge" LLM သို့ template နှင့်အတူ ပေးပို့ပါ။
4. Judge LLM သည် သင့် observability tool တွင် မှတ်တမ်းတင်ရန် အဆင့်သတ်မှတ်ချက် သို့မဟုတ် အမှတ်အသားဖြင့် ပြန်လည်ဖြေကြားပါမည်။

Langfuse မှ ဥပမာ:

![LLM-as-a-Judge Evaluator](https://langfuse.com/images/cookbook/example-autogen-evaluation/evaluator.png)


In [12]:
with langfuse.start_as_current_span(name="autogen-request-user-feedback-2") as span:

    async for message in team.run_stream(task="I am a picky eater and not sure if you find something for me."):
            if isinstance(message, TaskResult):
                print("Stop Reason:", message.stop_reason)
            else:
                print(message) 

    span.update_trace(
        input=user_input,
        output=message.stop_reason,
    )

langfuse.flush()

id='eefc628d-502f-451a-8f70-be486f62f8c5' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 29, 171393, tzinfo=datetime.timezone.utc) content='I am a picky eater and not sure if you find something for me.' type='TextMessage'
id='13b3e14b-bcf7-42a5-80d6-54b0c7be765e' source='meal_planner_agent' models_usage=RequestUsage(prompt_tokens=352, completion_tokens=27) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 30, 433516, tzinfo=datetime.timezone.utc) content='Chicken Alfredo Pasta  \n- Gluten-free pasta  \n- Grilled chicken breast  \n- Heavy cream  \n- Parmesan cheese  \n- Garlic  ' type='TextMessage'
id='550f2dee-0e08-4bbd-b67f-991b467328f1' source='nutritionist_agent' models_usage=RequestUsage(prompt_tokens=386, completion_tokens=17) metadata={} created_at=datetime.datetime(2025, 7, 2, 16, 38, 31, 505173, tzinfo=datetime.timezone.utc) content='Consider incorporating some vegetables, like spinach or broccoli, to increase the nutrien

ဤဥပမာ၏ဖြေကြားချက်ကို "အဆိပ်မရှိ" ဟုစီရင်ထားသည်ကို သင်ကြည့်နိုင်ပါသည်။

![LLM-as-a-Judge အကဲဖြတ်အဆင့်](https://langfuse.com/images/cookbook/example-autogen-evaluation/llm-as-a-judge-score.png)


#### ၅။ ကြည့်ရှုနိုင်မှု မီထရစ်များအကျဉ်းချုပ်

ဤမီထရစ်များအားလုံးကို dashboard များတွင်အတူတကွမြင်နိုင်ပါသည်။ ဒါက သင့် agent က session များစွာအတွင်းမှာ ဘယ်လိုလုပ်ဆောင်သလဲဆိုတာကို အလျင်အမြန်ကြည့်ရှုနိုင်စေပြီး အရည်အသွေးမီထရစ်များကို အချိန်ကြာလာတာနဲ့အမျှ ထိန်းသိမ်းနိုင်စေပါတယ်။

![ကြည့်ရှုနိုင်မှု မီထရစ်များအကျဉ်းချုပ်](https://langfuse.com/images/cookbook/example-autogen-evaluation/dashboard.png)


## အော့ဖ်လိုင်းအကဲဖြတ်ခြင်း

အွန်လိုင်းအကဲဖြတ်ခြင်းသည် တိုက်ရိုက်တုံ့ပြန်မှုအတွက် အရေးကြီးသော်လည်း **အော့ဖ်လိုင်းအကဲဖြတ်ခြင်း** လည်း လိုအပ်ပါသည်။ ၎င်းသည် ဖွံ့ဖြိုးတိုးတက်မှုမတိုင်မီ သို့မဟုတ် ဖွံ့ဖြိုးမှုအတွင်း စနစ်တကျ စစ်ဆေးမှုများ ပြုလုပ်ရန် အရေးကြီးပါသည်။ ၎င်းက အပြောင်းအလဲများကို ထုတ်လုပ်မှုအဆင့်သို့ ရောက်ရှိမည့်အချိန်တွင် အရည်အသွေးနှင့် ယုံကြည်စိတ်ချမှုကို ထိန်းသိမ်းနိုင်စေပါသည်။


### ဒေတာဆက်စပ်မှု အကဲဖြတ်ခြင်း

အော့ဖ်လိုင်းအကဲဖြတ်မှုတွင် သင်သည် အများအားဖြင့် အောက်ပါအဆင့်များကို လိုက်နာရမည်ဖြစ်သည် -
1. ပုံမှန်စမ်းသပ်ရန် ဒေတာစုပေါင်း (prompt နှင့် မျှော်မှန်းထားသော output အစုံအလင်များပါဝင်သည်) ရှိထားသည်။
2. အဆိုပါ ဒေတာစုပေါင်းပေါ်တွင် သင့် agent ကို အလုပ်လုပ်စေသည်။
3. ရလဒ်များကို မျှော်မှန်းထားသော ရလဒ်များနှင့် နှိုင်းယှဉ်သည်၊ သို့မဟုတ် အပိုအဆင့် အမှတ်ပေးစနစ်တစ်ခုကို အသုံးပြုသည်။

အောက်တွင် [q&a-dataset](https://huggingface.co/datasets/junzhang1207/search-dataset) ကို အသုံးပြု၍ ဤနည်းလမ်းကို ပြသထားပါသည်။ ဤဒေတာစုပေါင်းတွင် မေးခွန်းများနှင့် မျှော်မှန်းထားသော အဖြေများ ပါဝင်သည်။


In [16]:
import pandas as pd
from datasets import load_dataset
 
# Fetch search-dataset from Hugging Face
dataset = load_dataset("junzhang1207/search-dataset", split = "train")
df = pd.DataFrame(dataset)
print("First few rows of search-dataset:")
print(df.head())

/Users/jannik/Documents/GitHub/ai-agents-for-beginners/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


First few rows of search-dataset:
                                     id  \
0  20caf138-0c81-4ef9-be60-fe919e0d68d4   
1  1f37d9fd-1bcc-4f79-b004-bc0e1e944033   
2  76173a7f-d645-4e3e-8e0d-cca139e00ebe   
3  5f5ef4ca-91fe-4610-a8a9-e15b12e3c803   
4  64dbed0d-d91b-4acd-9a9c-0a7aa83115ec   

                                            question  \
0                 steve jobs statue location budapst   
1  Why is the Battle of Stalingrad considered a t...   
2  In what year did 'The Birth of a Nation' surpa...   
3  How many Russian soldiers surrendered to AFU i...   
4   What event led to the creation of Google Images?   

                                     expected_answer       category       area  
0  The Steve Jobs statue is located in Budapest, ...           Arts  Knowledge  
1  The Battle of Stalingrad is considered a turni...   General News       News  
2  This question is based on a false premise. 'Th...  Entertainment       News  
3  About 300 Russian soldiers surrendered to t

နောက်တစ်ဆင့်၊ ကျွန်ုပ်တို့သည် လောင်ဖျူးစ်တွင် အချက်အလက်စုစည်းမှု entity တစ်ခုကို ဖန်တီးပြီး အလုပ်လည်ပတ်မှုများကို ချက်ချင်းစောင့်ကြည့်မည်ဖြစ်သည်။ ထို့နောက်၊ အချက်အလက်စုစည်းမှုမှ အရာဝတ္ထုတစ်ခုစီကို စနစ်ထဲသို့ ထည့်သွင်းမည်။


In [17]:
from langfuse import Langfuse
langfuse = Langfuse()
 
langfuse_dataset_name = "qa-dataset_autogen-agent"
 
# Create a dataset in Langfuse
langfuse.create_dataset(
    name=langfuse_dataset_name,
    description="q&a dataset uploaded from Hugging Face",
    metadata={
        "date": "2025-03-21",
        "type": "benchmark"
    }
)

Dataset(id='cmcm7524d00kjad07s2cjwqcf', name='qa-dataset_autogen-agent', description='q&a dataset uploaded from Hugging Face', metadata={'date': '2025-03-21', 'type': 'benchmark'}, project_id='cloramnkj0002jz088vzn1ja4', created_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2025, 7, 2, 16, 54, 7, 357000, tzinfo=datetime.timezone.utc))

In [18]:
df_25 = df.sample(25) # For this example, we upload only 25 dataset questions

for idx, row in df_25.iterrows():
    langfuse.create_dataset_item(
        dataset_name=langfuse_dataset_name,
        input={"text": row["question"]},
        expected_output={"text": row["expected_answer"]}
    )

![Dataset items in Langfuse](https://langfuse.com/images/cookbook/example-autogen-evaluation/example-dataset.png)


#### ဒေတာစနစ်ပေါ်မှာ Agent ကို အလုပ်လုပ်စေခြင်း

ပထမဆုံး၊ Azure OpenAI မော်ဒယ်တွေကို အသုံးပြုပြီး မေးခွန်းတွေကို ဖြေဆိုပေးတဲ့ ရိုးရှင်းတဲ့ Autogen agent ကို တည်ဆောက်ပါ။


In [8]:
import os
from dotenv import load_dotenv

from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage

In [ ]:
load_dotenv()
client = AzureAIChatCompletionClient(
    model="gpt-4o",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.getenv("GITHUB_TOKEN")),
    max_tokens=5000,
    model_info={
        "json_output": True,
        "function_calling": False,
        "vision": False,
        "family": "unknown",
        "structured_output": True,
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

In [18]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[],
    system_message="You are participant in a quizz show and you are given a question. You need to create a short answer to the question.",
)

In [19]:
async def my_agent(user_query: str):

    with langfuse.start_as_current_span(name="autogen-trace") as span:

        # Execute the agent response
        response = await agent.on_messages(
            [TextMessage(content=user_query, source="user")],
            cancellation_token=CancellationToken(),
        )

        span.update_trace(
            input=user_query,
            output=response.chat_message.content,
        )

    return str(response.chat_message.content)

# Test the function
await my_agent("What is the capital of France?")

'The capital of France is Paris.'

အဆုံးတွင်၊ ကျွန်ုပ်တို့သည် ဒေတာစနစ်အပိုင်းတစ်ခုချင်းစီကို loop ပြုလုပ်ပြီး၊ agent ကို run ပြုလုပ်ကာ၊ trace ကို ဒေတာစနစ်အပိုင်းနှင့်ချိတ်ဆက်ပါသည်။ လိုအပ်ပါက အမြန်သုံးသပ်မှုအဆင့်ကိုလည်း ထည့်သွင်းနိုင်ပါသည်။


In [20]:
dataset_name = "qa-dataset_autogen-agent"
current_run_name = "dev_tasks_run-autogen_gpt-4.1" # Identifies this specific evaluation run
current_run_metadata={"model_provider": "Azure", "model": "gpt-4.1"}
current_run_description="Evaluation run for Autogen model on July 3rd"

dataset = langfuse.get_dataset('qa-dataset_autogen-agent')

for item in dataset.items:
    print(f"Running evaluation for item: {item.id} (Input: {item.input})")
 
    # Use the item.run() context manager
    with item.run(
        run_name=current_run_name,
        run_metadata=current_run_metadata,
        run_description=current_run_description
    ) as root_span: 
        # All subsequent langfuse operations within this block are part of this trace.
        generated_answer = await my_agent(user_query = item.input["text"])
    
    print("Generated Answer: ", generated_answer)
 
print(f"\nFinished processing dataset '{dataset_name}' for run '{current_run_name}'.")

langfuse.flush()

Running evaluation for item: 09810cc4-9992-4712-a3b2-7224da31776a (Input: {'text': 'In Hindu mythology, which deity is the Ganges river dolphin associated with?'})
Generated Answer:  In Hindu mythology, the Ganges river dolphin is associated with the deity Ganga.
Running evaluation for item: bb113f94-7723-47c6-8c34-59d883044514 (Input: {'text': 'What significant discovery did the LHCb collaboration report in 2015?'})
Generated Answer:  In 2015, the LHCb collaboration reported the discovery of pentaquark particles.
Running evaluation for item: 4d8ae54e-ceab-46d0-ad2c-6e8e223589a9 (Input: {'text': 'What is the MÄ\x81ori name for the red-crowned parakeet?'})
Generated Answer:  The Māori name for the red-crowned parakeet is kākāriki.
Running evaluation for item: 21e5a0d5-f619-4a73-868e-9955053b3e72 (Input: {'text': 'Who starred in the 1978 television film adaptation of Les MisÃ©rables?'})
Generated Answer:  Richard Jordan starred as Jean Valjean in the 1978 television film adaptation of Le

Markdown ဖိုင်ကို မြန်မာဘာသာ (my) သို့ ဘာသာပြန်ထားသည်မှာ အောက်ပါအတိုင်းဖြစ်ပါသည် -

သင်ဤလုပ်ငန်းစဉ်ကို အမျိုးမျိုးသော agent configuration များဖြင့် ထပ်မံလုပ်ဆောင်နိုင်ပါသည်၊ ဥပမာအားဖြင့် -

- မော်ဒယ်များ (gpt-4o-mini, gpt-4.1, စသည်တို့)
- Prompt များ
- Tools (ရှာဖွေမှု vs ရှာဖွေမှုမရှိမှု)
- Agent ရှုပ်ထွေးမှု (agent တစ်ခုတည်း vs agent များစွာ)

ထို့နောက် Langfuse တွင် အတူတူယှဉ်ကြည့်နိုင်ပါသည်။ ဤဥပမာတွင်၊ ကျွန်ုပ်သည် agent ကို 25 dataset မေးခွန်းများတွင် 3 ကြိမ်အထိ run လုပ်ခဲ့ပါသည်။ တစ်ခုချင်းစီ run လုပ်ရာတွင် မတူညီသော Azure OpenAI မော်ဒယ်ကို အသုံးပြုခဲ့ပါသည်။ မော်ဒယ်အရွယ်အစားကြီးလာသည်နှင့်အမျှ (မျှော်လင့်ထားသလို) မှန်ကန်သော မေးခွန်းများ၏ အရေအတွက်က တိုးလာသည်ကို တွေ့ရနိုင်ပါသည်။ `correct_answer` score ကို [LLM-as-a-Judge Evaluator](https://langfuse.com/docs/scores/model-based-evals) မှ ဖန်တီးထားပြီး၊ dataset တွင်ပေးထားသော sample answer အပေါ် မေးခွန်း၏ မှန်ကန်မှုကို စီရင်ရန် အဆင်ပြေစေရန် ဖန်တီးထားသည်။

![Dataset run overview](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset_runs.png)
![Dataset run comparison](https://langfuse.com/images/cookbook/example-autogen-evaluation/dataset-run-comparison.png)



---

**အကြောင်းကြားချက်**:  
ဤစာရွက်စာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ကို အသုံးပြု၍ ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှုအတွက် ကြိုးစားနေပါသော်လည်း၊ အလိုအလျောက် ဘာသာပြန်မှုများတွင် အမှားများ သို့မဟုတ် မမှန်ကန်မှုများ ပါဝင်နိုင်သည်ကို သတိပြုပါ။ မူရင်းဘာသာစကားဖြင့် ရေးသားထားသော စာရွက်စာတမ်းကို အာဏာတရ အရင်းအမြစ်အဖြစ် ရှုလေ့လာသင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်များမှ ပရော်ဖက်ရှင်နယ် ဘာသာပြန်မှုကို အကြံပြုပါသည်။ ဤဘာသာပြန်မှုကို အသုံးပြုခြင်းမှ ဖြစ်ပေါ်လာသော အလွဲအလွတ်များ သို့မဟုတ် အနားယူမှုများအတွက် ကျွန်ုပ်တို့သည် တာဝန်မယူပါ။
